In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%%capture
!pip install transformers[torch]
!pip install kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c detecting-french-texts-difficulty-level-2023
from zipfile import ZipFile
with ZipFile('detecting-french-texts-difficulty-level-2023.zip','r') as zip:
  zip.extractall(path="")

detecting-french-texts-difficulty-level-2023.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [ ]:
import torch

In [ ]:
unlabelled_data = pd.read_csv('unlabelled_test_data.csv', index_col = 'id')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
new_model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Data Science and Machine Learning/Project_Data_Science/train_model_bert_V2/').to(device)

In [ ]:
%%capture
new_tokenizer = AutoTokenizer.from_pretrained("camembert-base")

In [ ]:
def get_multiclass_predictions(texts):
    encodings = new_tokenizer(texts.tolist(), return_tensors="pt", padding="max_length", truncation=True, max_length=256)
    encodings = {k: v.to(new_model.device) for k, v in encodings.items()}

    with torch.no_grad():
        outputs = new_model(**encodings)

    logits = outputs.logits

    softmax = torch.nn.Softmax(dim=-1)  # Adjust the dimension to -1
    probabilities = softmax(logits.squeeze().cpu())
    probabilities = probabilities.detach().numpy()

    labels = np.argmax(probabilities, axis=-1)

    label_mapping = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}

    predicted_labels = [label_mapping[label.item()] for label in labels]

    return predicted_labels

In [ ]:
predicted_labels = get_multiclass_predictions(unlabelled_data['sentence'])

In [ ]:
unlabelled_data['difficulty'] = predicted_labels

In [ ]:
predictions = unlabelled_data[['difficulty']]

In [ ]:
predictions.head()

In [ ]:
predictions.to_csv('submission.csv')

In [ ]:
#! kaggle competitions submit -c detecting-french-texts-difficulty-level-2023 -f submission.csv -m "Sample submission"

## Test accuracy socre : 0.539
